In [11]:
import pandas as pd
import os
import re
from src.data.dataframe import createDataFrame,formatDataFrame
from sqlalchemy import create_engine


dir_name = "L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data"

#dataframe creation
device_df = pd.DataFrame(columns={'rack_num', 'Date_Time' 'Device', 'State'})

super_df = []

for root, dirs, files in os.walk(dir_name):
    for filename in files:
        rack_num = re.search(r'CF(.*?)\.', filename)[1]
        try:
            if 'Device' in filename:
                print(os.path.join(root, filename))
                df_temp = createDataFrame()
                super_df.append(df_temp)
        except (TypeError) as error:
            print(error)
            continue


    
device_df = formatDataFrame(super_df)
device_df = device_df[['rack_num', 'Date_Time', 'Device', 'State']]
device_df.drop_duplicates(
     subset=['Date_Time'], inplace=True)

%store device_df

L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data\CFMUW\2016\201604\CFMUW.Sensor Input_18_04_16_23_57.csv
L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data\CFMUW\2016\201604\CFMUW.Sensor Input_19_04_16_23_57.csv
L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data\CFMUW\2016\201604\CFMUW.Sensor Input_21_04_16_23_57.csv
L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data\CFMUW\2016\201604\CFMUW.Sensor Input_22_04_16_23_57.csv
L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data\CFMUW\2016\201604\CFMUW.Sensor Input_23_04_16_23_57.csv
L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data\CFMUW\2016\201604\CFMUW.Sensor Input_24_04_16_23_57.csv
L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data\CFMUW\2016\201604\CFMUW.Sensor Input_25_04_16_23_57.csv
L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Ra

KeyboardInterrupt: 